# 线性代数

In [1]:
import math
import time
import torch
import numpy as np

## 生成数据

In [4]:
def synthetic_data(w,b,num_examples):
    X = torch.normal(0,1,(num_examples,len(w)))
    y = torch.matmul(X,w) + b
    y += torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))

true_w = torch.tensor([-2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print('features' ,features[0],'\nlabels' ,labels[0])

features tensor([ 0.8197, -1.1259]) 
labels tensor([6.3816])
